In [5]:
from dotenv import load_dotenv
import os
from langchain_openai import ChatOpenAI

load_dotenv()

llm = ChatOpenAI(model="gpt-4o", temperature=0) 

In [2]:
from langchain.prompts import ChatPromptTemplate
SYSTEM_PROMPT = "You are an expert sommelier with extensive knowledge in wine, wine pairing, and the intricacies of food and beverage service. Your primary role is to assist users in selecting the best wines and pairing them perfectly with meals. You have a deep understanding of various wine regions, grape varieties, wine production methods, and current trends in the industry. You possess a refined palate, able to discern subtle flavors and characteristics in wines. Your advice is always clear, approachable, and tailored to each user’s preferences and specific dining context. You also educate users on wine appreciation, proper wine service, and the art of creating a harmonious dining experience. Your demeanor is professional, courteous, and passionate about wine culture, aiming to make each wine selection and pairing a memorable experience for users."
wine_query = "이 와인에 어울리는 요리에는 어떤 것들이 있을까요?"

chat_template = ChatPromptTemplate.from_messages(
  [
    ('system', SYSTEM_PROMPT),
    ('human', [ {'type': 'text', 'text':'{text}'},
                {'type':'image_url', 'image_url':  {'url': '{image_url}'}}])
  ]
)
chat_template

ChatPromptTemplate(input_variables=['image_url', 'text'], input_types={}, partial_variables={}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='You are an expert sommelier with extensive knowledge in wine, wine pairing, and the intricacies of food and beverage service. Your primary role is to assist users in selecting the best wines and pairing them perfectly with meals. You have a deep understanding of various wine regions, grape varieties, wine production methods, and current trends in the industry. You possess a refined palate, able to discern subtle flavors and characteristics in wines. Your advice is always clear, approachable, and tailored to each user’s preferences and specific dining context. You also educate users on wine appreciation, proper wine service, and the art of creating a harmonious dining experience. Your demeanor is professional, courteous, and passionate about wine culture, aiming to m

# recomment_dish_chain 용 랭체인 프롬프트

In [6]:
from langchain_core.output_parsers import StrOutputParser
output_parser = StrOutputParser()
chain = chat_template|llm|output_parser
response = chain.invoke(
  {
    'text': wine_query,
    'image_url': 'https://images.vivino.com/thumbs/Z90I3--JRKWlpMA8wdLY-Q_pb_x600.png'
  }
)
response

'Primitivo 와인은 일반적으로 풍부하고 과일 향이 강하며, 약간의 스파이시한 맛이 특징입니다. 이 와인에 어울리는 요리로는 다음과 같은 것들이 있습니다:\n\n1. **그릴드 스테이크**: 육즙이 풍부한 스테이크는 Primitivo의 풍부한 과일 향과 잘 어울립니다.\n\n2. **바비큐 립**: 달콤하고 매콤한 바비큐 소스가 Primitivo의 스파이시한 노트와 조화를 이룹니다.\n\n3. **라자냐**: 토마토 소스와 치즈가 듬뿍 들어간 라자냐는 와인의 풍미를 잘 받쳐줍니다.\n\n4. **양고기 요리**: 허브와 함께 구운 양고기는 Primitivo의 복합적인 맛과 잘 어울립니다.\n\n5. **치즈 플래터**: 특히 고다, 체다 같은 숙성 치즈와 잘 어울립니다.\n\n이 와인은 다양한 고기 요리와 잘 어울리며, 풍부한 맛을 가진 음식과 함께 즐기기에 좋습니다.'

In [7]:
def recommend_dishes_chain(query):
    
  chat_template = ChatPromptTemplate.from_messages(
    [
      ('system', SYSTEM_PROMPT),
      ('human', [ {'type': 'text', 'text':query['text']},
                  {'type':'image_url', 'image_url':  {'url': query['image_url']}}])
    ]
  )
  chain = chat_template|llm|output_parser
  return chain

In [8]:
query_1 = {
  'text':wine_query,
  'image_url': 'https://images.vivino.com/thumbs/Z90I3--JRKWlpMA8wdLY-Q_pb_x600.png'
}
rec_dish_chain = recommend_dishes_chain(query_1)

In [9]:
rec_dish_chain.invoke(query_1)

'Primitivo 와인은 일반적으로 풍부하고 과일 향이 강하며, 약간의 스파이시한 맛이 특징입니다. 이 와인에 어울리는 요리로는 다음과 같은 것들이 있습니다:\n\n1. **바비큐 요리**: 그릴에 구운 고기, 특히 소고기나 양고기와 잘 어울립니다.\n2. **토마토 소스 파스타**: 진한 토마토 소스가 들어간 파스타 요리와 잘 어울립니다.\n3. **피자**: 특히 페퍼로니나 소시지가 들어간 피자와 잘 맞습니다.\n4. **치즈**: 고다, 체다 같은 진한 맛의 치즈와 잘 어울립니다.\n5. **스튜**: 고기와 야채가 들어간 진한 스튜 요리와도 잘 어울립니다.\n\n이 와인의 풍부한 맛과 향이 요리의 맛을 한층 더 돋보이게 해줄 것입니다.'

In [10]:
from langchain_core.runnables import RunnableLambda
runnable = RunnableLambda(recommend_dishes_chain)
response = runnable.invoke(query_1)
response

'Primitivo 와인은 일반적으로 풍부하고 과일 향이 강하며, 약간의 스파이시한 맛이 특징입니다. 이 와인에 어울리는 요리로는 다음과 같은 것들이 있습니다:\n\n1. **그릴드 스테이크**: 육즙이 풍부한 스테이크는 Primitivo의 풍부한 과일 향과 잘 어울립니다.\n\n2. **바비큐 립**: 달콤하고 매콤한 바비큐 소스가 Primitivo의 스파이시한 노트와 조화를 이룹니다.\n\n3. **라자냐**: 토마토 소스와 치즈가 듬뿍 들어간 라자냐는 이 와인의 풍미를 잘 받쳐줍니다.\n\n4. **양고기 요리**: 허브와 함께 조리한 양고기는 Primitivo의 복합적인 맛과 잘 어울립니다.\n\n5. **치즈 플래터**: 특히 고다, 체다 같은 강한 맛의 치즈와 잘 어울립니다.\n\n이 와인은 다양한 고기 요리와 잘 어울리며, 풍부한 맛을 가진 음식과 함께 즐기기에 좋습니다.'

# describe_dish_flavor_chain 용 랭체인 프롬프트

In [12]:
def describe_dish_flavor_chain(query):
  #prompttemplate 
  # parser
  #chain
  return ''

In [13]:
dish_system_prompt = """
            Persona:
            As a flavor analysis system, I am equipped with a deep understanding of food ingredients, cooking methods, and sensory properties such as taste, texture, and aroma. I can assess and break down the flavor profiles of dishes by identifying the dominant tastes (sweet, sour, salty, bitter, umami) as well as subtler elements like spice levels, richness, freshness, and aftertaste. I am able to compare different foods based on their ingredients and cooking techniques, while also considering cultural influences and typical pairings. My goal is to provide a detailed analysis of a dish’s flavor profile to help users better understand what makes it unique or to aid in choosing complementary foods and drinks.

            Role:

            1. Flavor Identification: I analyze the dominant and secondary flavors of a dish, highlighting key taste elements such as sweetness, acidity, bitterness, saltiness, umami, and the presence of spices or herbs.
            2. Texture and Aroma Analysis: Beyond taste, I assess the mouthfeel and aroma of the dish, taking into account how texture (e.g., creamy, crunchy) and scents (e.g., smoky, floral) contribute to the overall experience.
            3. Ingredient Breakdown: I evaluate the role each ingredient plays in the dish’s flavor, including their impact on the dish's balance, richness, or intensity.
            4. Culinary Influence: I consider the cultural or regional influences that shape the dish, understanding how traditional cooking methods or unique ingredients affect the overall taste.
            5. Food and Drink Pairing: Based on the dish's flavor profile, I suggest complementary food or drink pairings that enhance or balance the dish’s qualities.

            Examples:

            - Dish Flavor Breakdown:
            For a butter garlic shrimp, I identify the richness from the butter, the pungent aroma of garlic, and the subtle sweetness of the shrimp. The dish balances richness with a touch of saltiness, and the soft, tender texture of the shrimp is complemented by the slight crispness from grilling.

            - Texture and Aroma Analysis:
            A creamy mushroom risotto has a smooth, velvety texture due to the creamy broth and butter. The earthy aroma from the mushrooms enhances the umami flavor, while a sprinkle of Parmesan adds a savory touch with a mild sharpness.

            - Ingredient Role Assessment:
            In a spicy Thai curry, the coconut milk provides a rich, creamy base, while the lemongrass and lime add freshness and citrus notes. The chilies bring the heat, and the balance between sweet, sour, and spicy elements creates a dynamic flavor profile.

            - Cultural Influence:
            A traditional Italian margherita pizza draws on the classic combination of fresh tomatoes, mozzarella, and basil. The simplicity of the ingredients allows the flavors to shine, with the tanginess of the tomato sauce balancing the richness of the cheese and the freshness of the basil.

            - Food Pairing Example:
            For a rich chocolate cake, I would recommend a sweet dessert wine like Port to complement the bitterness of the chocolate, or a light espresso to contrast the sweetness and enhance the richness of the dessert.
        """

In [14]:
def describe_dish_flavor_chain(query):
    # 이미지가 있는 경우와 없는 경우를 구분하여 처리
    if query.get("image_urls"):
        # 이미지가 있는 경우: 멀티모달 프롬프트
        messages = [
            ("system", dish_system_prompt),
            ("human", [ #"human" 으로 해도 무방 - 역할 태그의 차이가 크지 않음
                {"type": "text", "text": "이 요리의 이름과 맛을 한 문장으로 요약해주세요."},
                *[{"type": "image_url", "image_url": {"url": url}} for url in query["image_urls"]]
            ])
        ]
    else:
        # 텍스트만 있는 경우 (이미지 없이는 요리 설명이 어려우므로 안내 메시지)
        messages = [
            ("system", dish_system_prompt),
            ("user", "요리 이미지가 제공되지 않았습니다. 요리 이미지를 업로드해주시면 정확한 분석을 도와드리겠습니다.")
        ]
    
    prompt = ChatPromptTemplate.from_messages(messages)
    output_parser = StrOutputParser()
    
    chain = prompt | llm | output_parser
    return chain

In [ ]:
runnable = RunnableLambda(describe_dish_flavor_chain)
response = runnable.invoke({
    "image_urls": ["https://www.stockfood.com/Sites/StockFood/Documents/Homepage/News//en/16.jpg"]
})

print(response)

이 요리는 견과류와 귀리가 어우러진 그래놀라 바로, 고소하고 달콤한 맛이 특징입니다.


# wine_search

In [18]:
from dotenv import load_dotenv
import os
from langchain_openai import ChatOpenAI

load_dotenv()

True

In [19]:
from langchain_openai import OpenAIEmbeddings
from langchain_pinecone import PineconeVectorStore

embedding = OpenAIEmbeddings(model='text-embedding-3-small')
vector_store = PineconeVectorStore(
    index_name=os.getenv("PINECONE_INDEX_NAME"),
    embedding=embedding,
    pinecone_api_key=os.getenv("PINECONE_API_KEY")
)

In [20]:
# vector store에서 현재 요리의 풍미와 맛과 유사한 와인 검색
taste_query =  "이 요리는 판차넬라 샐러드로, 신선한 토마토와 바질의 상큼함이 빵의 고소함과 어우러져 상쾌하고 풍부한 맛을 냅니다."
results = vector_store.similarity_search(
  taste_query, 
    k=5, 
    namespace=os.getenv("PINECONE_NAMESPACE")
)

results

[Document(id='d5e69ead-ae12-4330-87b0-1219993d8d6d', metadata={'row': 12067.0, 'source': './winemag-data-130k-v2.csv'}, page_content=': 12067\ncountry: US\ndescription: Overtly sweet, this tastes like a dessert pastry turned into wine. The flavors are of orange, pineapple, vanilla-cream and buttered toast.\ndesignation: Back Seat Blonde\npoints: 82\nprice: 14.0\nprovince: California\nregion_1: Santa Ynez Valley\nregion_2: Central Coast\ntaster_name: \ntaster_twitter_handle: \ntitle: Coquelicot 2010 Back Seat Blonde White (Santa Ynez Valley)\nvariety: White Blend\nwinery: Coquelicot'),
 Document(id='3984eddd-0c7d-4da9-b036-e3522cbb0f12', metadata={'row': 12067.0, 'source': './winemag-data-130k-v2.csv'}, page_content=': 12067\ncountry: US\ndescription: Overtly sweet, this tastes like a dessert pastry turned into wine. The flavors are of orange, pineapple, vanilla-cream and buttered toast.\ndesignation: Back Seat Blonde\npoints: 82\nprice: 14.0\nprovince: California\nregion_1: Santa Ynez 

In [21]:
results = vector_store.similarity_search(
   response, 
    k=5, 
    namespace=os.getenv("PINECONE_NAMESPACE")
)

results

[Document(id='af81b512-2a9d-468b-9ee2-6ebf9356f376', metadata={'row': 108411.0, 'source': './winemag-data-130k-v2.csv'}, page_content=": 108411\ncountry: US\ndescription: A sweet smell of honeysuckle and grapefruit candy permeates the nose of this bottling, along with cut honeydew melon and apple blossom. Sugary mandarin juice is the primary flavor on the palate, but it's properly offset by acidity a chalky texture.\ndesignation: \npoints: 87\nprice: 29.0\nprovince: California\nregion_1: Central Coast\nregion_2: \ntaster_name: Matt Kettmann\ntaster_twitter_handle: @mattkettmann\ntitle: Coquelicot 2016 Riesling\nvariety: Riesling\nwinery: Coquelicot"),
 Document(id='7e507d94-9c2a-4637-b164-05e6d65a7836', metadata={'row': 108411.0, 'source': './winemag-data-130k-v2.csv'}, page_content=": 108411\ncountry: US\ndescription: A sweet smell of honeysuckle and grapefruit candy permeates the nose of this bottling, along with cut honeydew melon and apple blossom. Sugary mandarin juice is the prim

In [22]:
def search_wine(dish_flavor):
    results = vector_store.similarity_search(
        dish_flavor, 
        k=5, 
        namespace=os.getenv("PINECONE_NAMESPACE")
    )

    return {
        "dish_flavor": dish_flavor,
        "wine_reviews": "\n".join([doc.page_content for doc in results])
    }

In [ ]:
runnable = RunnableLambda(search_wine)
response = runnable.invoke(taste_query)
print(response['dish_flavor'])
print(response['wine_reviews'])

이 요리는 판차넬라 샐러드로, 신선한 토마토와 바질의 상큼함이 빵의 고소함과 어우러져 상쾌하고 풍부한 맛을 냅니다.
: 12067
country: US
description: Overtly sweet, this tastes like a dessert pastry turned into wine. The flavors are of orange, pineapple, vanilla-cream and buttered toast.
designation: Back Seat Blonde
points: 82
price: 14.0
province: California
region_1: Santa Ynez Valley
region_2: Central Coast
taster_name: 
taster_twitter_handle: 
title: Coquelicot 2010 Back Seat Blonde White (Santa Ynez Valley)
variety: White Blend
winery: Coquelicot
: 12067
country: US
description: Overtly sweet, this tastes like a dessert pastry turned into wine. The flavors are of orange, pineapple, vanilla-cream and buttered toast.
designation: Back Seat Blonde
points: 82
price: 14.0
province: California
region_1: Santa Ynez Valley
region_2: Central Coast
taster_name: 
taster_twitter_handle: 
title: Coquelicot 2010 Back Seat Blonde White (Santa Ynez Valley)
variety: White Blend
winery: Coquelicot
: 115812
country: South Africa
description: Vanilla

In [28]:
from pinecone import Pinecone
pc = Pinecone(api_key=os.getenv("PINECONE_API_KEY"))

In [29]:
# 기존 인덱스 정보 확인 (차원수 등)
existing_index = pc.Index(os.getenv("PINECONE_INDEX_NAME"))
index_stats = existing_index.describe_index_stats()

In [30]:
index_stats

{'dimension': 1536,
 'index_fullness': 0.0,
 'metric': 'cosine',
 'namespaces': {'wine-reviews-ns1': {'vector_count': 259942},
                'wine_description_only': {'vector_count': 130331}},
 'total_vector_count': 390273,
 'vector_type': 'dense'}

In [31]:
query = "달콤한 맛을 느낄 수 있는 와인"

results = vector_store.similarity_search(
    query, 
    k=5, 
    namespace="wine_description_only"
)

results

[Document(id='9e01a8c3-88cd-4ffa-b606-6158fd44f255', metadata={'country': 'US', 'points': 94.0, 'price': 105.0, 'row': 45019.0, 'title': 'Adelsheim 2013 Winderlea Vineyard Pinot Noir (Dundee Hills)', 'variety': 'Pinot Noir'}, page_content="Once again this designate amazes with a surfeit of beautifully integrated flavors. Mushrooms and baking spices adorn an explosion of strawberry pastries. There's plenty of ripe cherry fruit also, and a long, polished, lip-licking finish."),
 Document(id='1235b0ec-e642-42db-a06a-9401c7a60fd9', metadata={'country': 'US', 'points': 94.0, 'price': 50.0, 'row': 59514.0, 'title': 'De Loach 2010 Maboroshi Vineyard Pinot Noir (Russian River Valley)', 'variety': 'Pinot Noir'}, page_content="Dazzling with silky richness, this wine is so delicious, you can't help but to fall in love with it. Tantalizing with waves of raspberry and cherry pie, ripe persimmon, mocha, red licorice and smoky oak flavors that finish long and satisfying. The balance of acids and tann

In [32]:
# 국가별 필터링
def search_wine_by_country(query, country="US"):
    results = vector_store.similarity_search(
        query,
        k=5,
        filter={"country": country},
        namespace="wine_description_only"
    )
    return results

In [33]:
search_wine_by_country(query)

[Document(id='9e01a8c3-88cd-4ffa-b606-6158fd44f255', metadata={'country': 'US', 'points': 94.0, 'price': 105.0, 'row': 45019.0, 'title': 'Adelsheim 2013 Winderlea Vineyard Pinot Noir (Dundee Hills)', 'variety': 'Pinot Noir'}, page_content="Once again this designate amazes with a surfeit of beautifully integrated flavors. Mushrooms and baking spices adorn an explosion of strawberry pastries. There's plenty of ripe cherry fruit also, and a long, polished, lip-licking finish."),
 Document(id='1235b0ec-e642-42db-a06a-9401c7a60fd9', metadata={'country': 'US', 'points': 94.0, 'price': 50.0, 'row': 59514.0, 'title': 'De Loach 2010 Maboroshi Vineyard Pinot Noir (Russian River Valley)', 'variety': 'Pinot Noir'}, page_content="Dazzling with silky richness, this wine is so delicious, you can't help but to fall in love with it. Tantalizing with waves of raspberry and cherry pie, ripe persimmon, mocha, red licorice and smoky oak flavors that finish long and satisfying. The balance of acids and tann